In [2]:
import os
import torch
import torch.nn as nn
import torchvision.utils as vutils
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
from PIL import Image

class ModelEvaluator:
    """
    Clase para evaluar y visualizar resultados del GAN
    """
    def __init__(self, generator_path, latent_dim=100, image_size=64):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.latent_dim = latent_dim
        self.image_size = image_size
        
        # Cargar generador
        self.generator = Generator().to(self.device)
        self.generator.load_state_dict(torch.load(generator_path))
        self.generator.eval()
        
    def generate_samples(self, num_samples=16, show=True):
        """
        Genera y muestra muestras aleatorias
        """
        with torch.no_grad():
            # Generar ruido aleatorio
            noise = torch.randn(num_samples, self.latent_dim, device=self.device)
            fake_images = self.generator(noise)
            
            # Convertir imágenes para visualización
            img_grid = vutils.make_grid(fake_images, padding=2, normalize=True)
            img_grid = img_grid.cpu().numpy().transpose((1, 2, 0))
            
            if show:
                plt.figure(figsize=(8, 8))
                plt.axis("off")
                plt.title("Muestras Generadas")
                plt.imshow(img_grid)
                plt.show()
            
            return fake_images
    
    def interpolate_latent_space(self, num_steps=10):
        """
        Genera interpolación entre dos puntos en el espacio latente
        """
        with torch.no_grad():
            # Generar dos vectores latentes aleatorios
            z1 = torch.randn(1, self.latent_dim, device=self.device)
            z2 = torch.randn(1, self.latent_dim, device=self.device)
            
            # Crear interpolaciones
            alphas = np.linspace(0, 1, num_steps)
            interpolated_images = []
            
            for alpha in alphas:
                z_interp = z1 * (1 - alpha) + z2 * alpha
                fake_image = self.generator(z_interp)
                interpolated_images.append(fake_image)
            
            # Concatenar y mostrar resultados
            interpolated_images = torch.cat(interpolated_images, dim=0)
            img_grid = vutils.make_grid(interpolated_images, nrow=num_steps, padding=2, normalize=True)
            img_grid = img_grid.cpu().numpy().transpose((1, 2, 0))
            
            plt.figure(figsize=(15, 4))
            plt.axis("off")
            plt.title("Interpolación en el Espacio Latente")
            plt.imshow(img_grid)
            plt.show()
            
            return interpolated_images
    
    def style_mixing(self, num_samples=4):
        """
        Genera mezclas de estilo entre diferentes muestras
        """
        with torch.no_grad():
            # Generar vectores latentes
            z1 = torch.randn(num_samples, self.latent_dim, device=self.device)
            z2 = torch.randn(num_samples, self.latent_dim, device=self.device)
            
            # Crear matriz de mezclas
            mixed_images = []
            
            # Agregar imágenes originales
            mixed_images.append(self.generator(z1))
            
            # Crear mezclas con diferentes proporciones
            alphas = [0.25, 0.5, 0.75]
            for alpha in alphas:
                z_mixed = z1 * (1 - alpha) + z2 * alpha
                mixed_images.append(self.generator(z_mixed))
            
            # Agregar segundas imágenes originales
            mixed_images.append(self.generator(z2))
            
            # Mostrar resultados
            mixed_images = torch.cat(mixed_images, dim=0)
            img_grid = vutils.make_grid(mixed_images, nrow=num_samples, padding=2, normalize=True)
            img_grid = img_grid.cpu().numpy().transpose((1, 2, 0))
            
            plt.figure(figsize=(15, 15))
            plt.axis("off")
            plt.title("Mezcla de Estilos")
            plt.imshow(img_grid)
            plt.show()
            
            return mixed_images
    
    def save_image_grid(self, images, path, nrow=8):
        """
        Guarda una cuadrícula de imágenes en disco
        """
        vutils.save_image(images, path, nrow=nrow, padding=2, normalize=True)

def evaluate_model(generator_path, output_dir='./evaluation_results'):
    """
    Realiza una evaluación completa del modelo
    """
    # Crear directorio para resultados
    os.makedirs(output_dir, exist_ok=True)
    
    # Inicializar evaluador
    evaluator = ModelEvaluator(generator_path)
    
    print("Generando resultados...")
    
    # 1. Generar y guardar muestras aleatorias
    samples = evaluator.generate_samples(num_samples=64)
    evaluator.save_image_grid(
        samples,
        f'{output_dir}/random_samples.png',
        nrow=8
    )
    
    # 2. Generar y guardar interpolaciones
    interpolations = evaluator.interpolate_latent_space(num_steps=12)
    evaluator.save_image_grid(
        interpolations,
        f'{output_dir}/interpolations.png',
        nrow=12
    )
    
    # 3. Generar y guardar mezclas de estilo
    style_mixes = evaluator.style_mixing(num_samples=6)
    evaluator.save_image_grid(
        style_mixes,
        f'{output_dir}/style_mixing.png',
        nrow=6
    )
    
    print("Evaluación completada. Resultados guardados en:", output_dir)

# Ejemplo de uso
if __name__ == "__main__":
    # Ruta al modelo entrenado
    generator_path = './saved_models/generator_final.pt'
    
    # Realizar evaluación
    evaluate_model(generator_path)
    
    # Para evaluación interactiva
    evaluator = ModelEvaluator(generator_path)
    
    # Generar diferentes visualizaciones
    print("Generando muestras aleatorias...")
    evaluator.generate_samples(num_samples=16)
    
    print("\nGenerando interpolación en espacio latente...")
    evaluator.interpolate_latent_space(num_steps=10)
    
    print("\nGenerando mezclas de estilo...")
    evaluator.style_mixing(num_samples=4)

NameError: name 'Generator' is not defined